In [ ]:
import mlflow
import os


mlflow.set_tracking_uri('http://localhost:5000')
oputna_dir = '../../optuna'

### Train Model on single cripto

#### Train Baseline Model

In [ ]:
mlflow.set_experiment('Training_BASELINE')

In [ ]:
from common import get_dataframe, register_training_experiment

import pandas as pd

df = get_dataframe()

for coin in df.columns[1:]:
    df_coin = pd.DataFrame({'Date': df['Date'], coin: df[coin]})
    df_coin['lag1'] = df_coin[coin].shift(1)
    df_coin.dropna(inplace=True)
    
    register_training_experiment(
        data=df_coin[coin],
        predictions=df_coin['lag1'],
        model_name="BASELINE",
        coin=coin,
        x_axis=df_coin['Date'])

#### Train LSTM / RNN Model

In [ ]:
mlflow.set_experiment('Training_LSTM_RNN')
storage_name = f"sqlite:///{os.path.join(oputna_dir, 'training.db')}"

In [ ]:
import os

import numpy as np
from common import get_dataframe, evaluate_best_coin, objective

import mlflow
import optuna

df = get_dataframe()


for coin in df.iloc[:, 1:]:
    data = np.array(df[coin]).reshape(-1, 1)

    # Create an Optuna study
    study_name = coin  # Unique identifier of the study.
    
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)

    study = optuna.create_study(study_name=coin, direction='minimize', storage=storage_name, pruner=pruner, load_if_exists=True)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, coin), n_trials=100)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(coin, data, best_params)
    except ValueError:
        print("No best hyperparameters found")

#### Train LSTM / RNN Model with SMA

In [ ]:
mlflow.set_experiment('Training_LSTM_RNN_SMA')
storage_name = f"sqlite:///{os.path.join(oputna_dir, 'training_sma.db')}"

In [ ]:
import os

import numpy as np
from common import get_dataframe, evaluate_best_coin, objective

import optuna

df = get_dataframe(add_sma_columns=True)
criptos = df.filter(like='close', axis=1).columns
output_shape = 1

for cripto in criptos:

    coin = cripto.split('_')[1]
    cripto_data = df.filter(like=coin, axis=1)

    data = np.array(cripto_data)

    # Create an Optuna study
    study_name = coin  # Unique identifier of the study.
    pruner = optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=10)
    study = optuna.create_study(study_name=coin, direction='minimize', storage=storage_name, pruner=pruner)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, coin, output_shape), n_trials=100)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(coin, data, best_params, output_shape)
    except ValueError:
        print("No best hyperparameters found")

### Train Model on cripto clusters

#### Train LSTM / RNN Model

In [ ]:
mlflow.set_experiment('Training_LSTM_RNN_Cluster')
storage_name = f"sqlite:///{os.path.join(oputna_dir, 'training_cluster.db')}"

In [ ]:
import os
import numpy as np
from common import (evaluate_best_coin, get_clustered_dataframes, objective)
import mlflow
import optuna


clusters_data = get_clustered_dataframes()

for cluster, data in clusters_data.items():
    print(f"Cluster {cluster}")
    data = data.drop(columns=['Date'])
    criptos = ','.join(data.columns)
    data = np.array(data)

    study_name = f"Cluster_{cluster}"  # Unique identifier of the study.
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    study = optuna.create_study(study_name=study_name, direction='minimize', storage=storage_name, pruner=pruner)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, criptos), n_trials=100)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(criptos, data, best_params)
    except ValueError:
        print("No best hyperparameters found")

#### Train LSTM / RNN Model with SMA

In [ ]:
mlflow.set_experiment('Training_LSTM_RNN_Cluster_SMA')
storage_name = f"sqlite:///{os.path.join(oputna_dir, 'training_cluster_sma.db')}"

In [ ]:
import numpy as np
from common import get_clustered_dataframes, evaluate_best_coin, objective
import optuna


# Normalize the data to the range [0, 1] to help the LSTM model converge faster

clusters_data = get_clustered_dataframes(add_sma_columns=True)

for cluster, df in clusters_data.items():

    print(f"Cluster {cluster}")
    criptos = df.filter(like='close', axis=1).columns
    output_shape = len(criptos)
    criptos = ','.join([cripto.split("_")[1] for cripto in criptos])

    df = df.drop('Date', axis=1)
    data = np.array(df)

    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    study = optuna.create_study(study_name=criptos, direction='minimize', storage=storage_name, pruner=pruner)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, criptos, output_shape), n_trials=100)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(criptos, data, best_params, output_shape)
    except ValueError:
        print("No best hyperparameters found")

#### Train VAR

In [ ]:
mlflow.set_experiment('Training_VAR')
storage_name = f"sqlite:///{os.path.join(oputna_dir, 'training_var.db')}"

In [ ]:
import numpy as np
from common import get_clustered_dataframes
from common_var import evaluate_best_coin, objective
import optuna


# Normalize the data to the range [0, 1] to help the LSTM model converge faster

clusters_data = get_clustered_dataframes()

for cluster, data in clusters_data.items():

    print(f"Cluster {cluster}")
    criptos = data.filter(like='close', axis=1).columns
    output_shape = len(criptos)
    criptos = ','.join([cripto.split("_")[1] for cripto in criptos])

    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    study = optuna.create_study(study_name=criptos, direction='minimize', storage=storage_name, pruner=pruner)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, criptos, output_shape), n_trials=100)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(criptos, data, best_params, output_shape)
    except ValueError:
        print("No best hyperparameters found")